## Modelo: **GPT-4o-mini**
---

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

MODEL = "gpt-4o-mini"

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
embeddings = OpenAIEmbeddings()  #(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

### Prompt template

In [11]:
from langchain.prompts import PromptTemplate

template = """
Eres un representatne de servicio al cliente, responde la pregunta basado en el contexto de abajo. 
Si no puedes responder la pregunta, responde "No lo se"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("docs/manual-lpcr.pdf")
pages = loader.load_and_split()

In [13]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
    )

# Instantiate a retriever
retriever = vectorstore.as_retriever()

In [14]:
from langchain_core.runnables import RunnablePassthrough

md_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
}
| prompt
| model
)

In [ ]:
question = "Como peso un producto?"

print(f"Pregunta: {question}")
md_chain.invoke({"context":"context" ,"question": "Como peso un producto?"})